<a href="https://colab.research.google.com/github/fumio125/ou_dip/blob/master/ou_dip_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
# imgをrefに張り合わせることを考える
ref = cv2.imread("pano_ref.jpg") # ベースとなる画像（BGR）
src = cv2.imread("pano_src.jpg") # 変換する画像（BGR）

# 参考： https://docs.opencv.org/master/db/d27/tutorial_py_table_of_contents_feature2d.html

# ORB を使う場合の例。AKAZEならAKAZE_create
## BRIEFなら keypointがStarDetector_create (FAST特徴点)、descriptorがBriefDescriptorExtractor_create()
## SIFTは、特許関係の問題で、nonfreeオプションをONにしてビルドしないと入りません
## いまや、あえて使う必要はないかもしれません（精度ならAKAZE、速さならORBがおすすめ）
## インストール済みの場合：cv2.xfeatures2d.SIFT_create() で使えます。Colabにはない
orb = cv2.ORB_create()

# 各画像に対するkeypointとdescriptorの計算
kp_ref, des_ref = orb.detectAndCompute(ref, None)
kp_src, des_src = orb.detectAndCompute(src, None)

# マッチング。ORBならハミング距離をつかうべき
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(des_ref,des_src)

# 距離が小さい順に並べてみる（可視化のため）
matches = sorted(matches, key = lambda x:x.distance)

# 最も小さい距離で対応づいた100組の対応点を可視化してみる
corr_disp = cv2.drawMatches(ref,kp_ref,src,kp_src,matches[:100],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2_imshow(corr_disp) # 表示

# 対応点の登録とホモグラフィ行列の推定（w/ RANSAC。閾値は5.0画素にした）
pts_ref = np.float32([ kp_ref[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
pts_src = np.float32([ kp_src[m.trainIdx].pt  for m in matches]).reshape(-1,1,2)
H, mask = cv2.findHomography(pts_src, pts_ref, cv2.RANSAC,5.0)
